In [1]:
pip install yt-dlp openai-whisper transformers torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 15.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB

In [7]:
import os
import yt_dlp
import whisper
from transformers import pipeline
from transformers import MarianMTModel, MarianTokenizer

def download_audio(youtube_url, output_path='downloads'):
    """Download audio from YouTube video."""
    os.makedirs(output_path, exist_ok=True)
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_path, '%(title)s.%(ext)s'),
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=True)
        audio_path = ydl.prepare_filename(info).replace('.webm', '.mp3').replace('.m4a', '.mp3')
    return audio_path

def transcribe_audio(audio_path, model='base', language='en'):
    """Transcribe audio using OpenAI Whisper."""
    whisper_model = whisper.load_model(model)
    result = whisper_model.transcribe(audio_path, language=language)
    return result['text']

def translate_text(text, src_lang='en', tgt_lang='fr'):
    """Translate text to target language using MarianMT, with validation."""
    available_models = [
        'en-fr', 'en-es', 'en-de', 'en-it', 'en-ru', 'en-zh', 'fr-en', 'es-en', 'de-en', 'it-en', 'ru-en', 'zh-en'
    ]

    if src_lang == tgt_lang:
        print("Source and target language are the same. Skipping translation.")
        return text  # Return original text if translation is unnecessary

    model_name = f'Helsinki-NLP/opus-mt-{src_lang}-{tgt_lang}'

    if f"{src_lang}-{tgt_lang}" not in available_models:
        print(f"Error: Translation from {src_lang} to {tgt_lang} is not supported.")
        return text  # Return original text if translation is not possible

    try:
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)

        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
        translated_tokens = model.generate(**inputs)
        translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]

        return translated_text
    except Exception as e:
        print(f"Translation failed: {e}")
        return text

def summarize_text(text, model_name='facebook/bart-large-cnn', chunk_size=1000):
    """Summarize transcribed text in chunks to avoid input length issues."""
    summarizer = pipeline('summarization', model=model_name)
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk, max_length=200, min_length=50, do_sample=False)
        summaries.append(summary[0]['summary_text'])

    return " ".join(summaries)  # Combine all summary chunks

def analyze_safety(summary):
    """Analyze if the summary is safe for children."""
    classifier = pipeline("text-classification", model="facebook/roberta-hate-speech-dynabench-r4-target")
    result = classifier(summary)
    label = result[0]['label']

    if label in ['not_hate', 'neutral']:
        return "Safe for children"
    else:
        return "Not safe for children"

if __name__ == "__main__":
    youtube_url = input("Enter YouTube video URL: ")
    target_language = input("Enter target language code (e.g., 'fr' for French, 'es' for Spanish, 'de' for German, etc.): ")

    audio_path = download_audio(youtube_url)
    print("Audio downloaded successfully.")

    transcript = transcribe_audio(audio_path)
    print("Transcription completed.")

    summary = summarize_text(transcript)
    print("Summary:")
    print(summary)

    translated_summary = translate_text(summary, src_lang='en', tgt_lang=target_language)
    print("Translated Summary:")
    print(translated_summary)

    safety_status = analyze_safety(summary)
    print(safety_status)


Enter YouTube video URL: https://www.youtube.com/shorts/ymf3C5NlFa0
Enter target language code (e.g., 'fr' for French, 'es' for Spanish, 'de' for German, etc.): en
[youtube] Extracting URL: https://www.youtube.com/shorts/ymf3C5NlFa0
[youtube] ymf3C5NlFa0: Downloading webpage
[youtube] ymf3C5NlFa0: Downloading tv client config
[youtube] ymf3C5NlFa0: Downloading player 19d2ae9d
[youtube] ymf3C5NlFa0: Downloading tv player API JSON
[youtube] ymf3C5NlFa0: Downloading ios player API JSON
[youtube] ymf3C5NlFa0: Downloading m3u8 information
[info] ymf3C5NlFa0: Downloading 1 format(s): 251
[download] Destination: downloads/SATRANGA SONG STATUS 🥺❤️ #animalmovie #arijitsinghsong #songstatus #foryou #aesthetic #fyp #shorts.webm
[download] 100% of  416.50KiB in 00:00:00 at 2.77MiB/s   
[ExtractAudio] Destination: downloads/SATRANGA SONG STATUS 🥺❤️ #animalmovie #arijitsinghsong #songstatus #foryou #aesthetic #fyp #shorts.mp3
Deleting original file downloads/SATRANGA SONG STATUS 🥺❤️ #animalmovie #ar

/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)
/usr/local/li

Transcription completed.


Device set to use cpu
Your max_length is set to 200, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


Summary:
So So So So.  So So so. So so so so So. So Soso so soso sososo. Soso Sosososo sooosoo sosoosoo. Soososoo! Sosooo!
Source and target language are the same. Skipping translation.
Translated Summary:
So So So So.  So So so. So so so so So. So Soso so soso sososo. Soso Sosososo sooosoo sosoosoo. Soososoo! Sosooo!


Device set to use cpu


Not safe for children
